In [ ]:
#pip install python_dotenv-0.20.0-py3-none-any.whl

In [ ]:
from imap_tools import MailBox
import pandas as pd
from dotenv import load_dotenv
import os
from bs4 import BeautifulSoup

In [ ]:
load_dotenv("login.env")

In [ ]:
login_address = os.environ.get("GMAIL_USER0")
password=os.environ.get("GMAIL_PASSWORD0")
folder='INBOX'

In [ ]:
with MailBox('imap.gmail.com').login(login_address, password, folder) as mailbox:
    allRows = []
    for msg in mailbox.fetch(mark_seen=False):
        print(f"{msg.uid} is processing")
        msgID = "A" + msg.uid
        text=""
        soup = BeautifulSoup(msg.html, features="html.parser")
        for script in soup(["script", "style"]):
            script.extract()    
        text = soup.get_text()
        if len(text)==0:
            text=msg.text
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk for chunk in chunks if chunk)
        new_text = (text[:32500] + '..') if len(text) > 32500 else text
        
        allRows.append(
            {"ID": msgID, "body": new_text, "to": msg.to, "from_": msg.from_})

In [86]:
df = pd.DataFrame(allRows, columns=["ID", "body", "to", "from_"])
df

,ID,body,to,from_
0,A1,"Welcome to your Google Account\nHi Dmml,\nWelc...","(dmml2test@gmail.com,)",googlecommunityteam-noreply@google.com
1,A2,"Your impartial, comprehensive news source.\nHe...","(dmml2test@gmail.com,)",DailyDigest@email.join1440.com
2,A3,Welcome to The Morning newsletter\nWe'll bring...,"(dmml2test@gmail.com,)",nytdirect@nytimes.com
3,A4,Thanks for signing up for Morning Brew\nToday ...,"(dmml2test@gmail.com,)",crew@morningbrew.com
4,A5,NextDraft\nImportant: Confirm Your Subscriptio...,"(dmml2test@gmail.com,)",dave@davenetics.com
...,...,...,...,...
3060,A3068,Hi [Name]\nI’m a big fan of your company. You ...,"(nciarunprasath@gmail.com,)",noemailid111@gmail.com
3061,A3069,Hi [Name]\nI have been following you on [socia...,"(nciarunprasath@gmail.com,)",noemailid111@gmail.com
3062,A3070,Hi [Name]\nThank you for signing up for the fr...,"(dmml2test@gmail.com,)",santhoshsrv45@gmail.com
3063,A3071,Hi [Name]\nThank you so much for being a custo...,"(nciarunprasath@gmail.com,)",noemailid111@gmail.com


In [87]:
df.to_csv('MailBodyOutput_latest.csv', index=False, encoding="utf-8-sig")